In [113]:
import os
# Path on master/worker where Python is installed
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3.6'

In [130]:
t0_df = spark.read.csv("sample-data/train_sample.csv",header=True,inferSchema=True)
t0_df.count()

Py4JJavaError: An error occurred while calling o1380.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 92.0 failed 1 times, most recent failure: Lost task 0.0 in stage 92.0 (TID 97, localhost, executor driver): java.io.FileNotFoundException: C:\Users\tabarlow\AppData\Local\Temp\blockmgr-56462a77-a201-4118-a57b-997a2ad1396e\26\temp_shuffle_7aa366e1-838c-4b60-8c18-6a2f02c3e364 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeS

In [125]:
t0_df = t0_df.sample(False,0.1)
t0_df.take(1)

[]

In [120]:
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

final_df = t0_df

vectorAssembler = VectorAssembler(inputCols=[
    "ip",
    "app",
    "device",
    "os",
    "channel"
    #"click_time", TimeStamp is not supported. Need to get important data from these
    #"attributed_time"
], outputCol="features")

v_final_df = vectorAssembler.transform(final_df)


Py4JJavaError: An error occurred while calling o1165.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 70.0 failed 1 times, most recent failure: Lost task 0.0 in stage 70.0 (TID 79, localhost, executor driver): java.io.FileNotFoundException: C:\Users\tabarlow\AppData\Local\Temp\blockmgr-56462a77-a201-4118-a57b-997a2ad1396e\1b\shuffle_7_0_0.data.2eb4fce1-7f2a-4cf4-8534-b9cf8142ab28 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.writePartitionedFile(BypassMergeSortShuffleWriter.java:191)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:163)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:96)
	at org.apache.spark.scheduler.ShuffleM

In [117]:
splits = v_final_df.randomSplit([0.6,0.4],1)

train_df = splits[0]
test_df = splits[1]

In [118]:
train_df.repartition(8)

DataFrame[ip: int, app: int, device: int, os: int, channel: int, click_time: timestamp, attributed_time: timestamp, is_attributed: int, features: vector]

In [119]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

dt = DecisionTreeClassifier(labelCol="is_attributed",featuresCol="features")

dt_model = dt.fit(train_df)

predictions_df = dt_model.transform(test_df)
predictions_df.take(1)

Py4JJavaError: An error occurred while calling o1069.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 68.0 failed 1 times, most recent failure: Lost task 0.0 in stage 68.0 (TID 78, localhost, executor driver): java.io.FileNotFoundException: C:\Users\tabarlow\AppData\Local\Temp\blockmgr-56462a77-a201-4118-a57b-997a2ad1396e\01\temp_shuffle_a8c026b1-7909-430e-a88a-a28035f20090 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSor

In [ ]:
#test_df = spark.read.csv("sample-data/test.csv",header=True,inferSchema=True)
